In [23]:
import os 
import numpy as np
import pandas as pd
from scipy.optimize import least_squares
from mpl_toolkits.mplot3d import Axes3D

import matplotlib.pyplot as plt

# Constants
g = 9.81  # Gravity (m/s²)
M = 9     # Minimum number of orientations (≥9) [x=0,x=180,y=0,y=180,z=0,z=180,xz=15,xz=20,xz=30]  
N = 480    # Samples per orientation


# Loading Data

In [32]:
def load_data(file_path):
    df = pd.read_csv(file_path)

    accel_data = df.iloc[:, 1:4].values # convert to m/s^2

    return accel_data


def load_all_data(folder_path):
    accel_list= []
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".csv"):
            file_path = os.path.join(folder_path, file_name)
            accel_data = load_data(file_path)
            accel_data= accel_data[:N]
            accel_list.append(accel_data)

    # Merge all extracted data
    accel_data = np.vstack(accel_list)

    invalid_indices =  np.where(np.isnan(accel_data))
    accel_data = np.delete(accel_data, invalid_indices, axis=0)
    return accel_data
data =load_all_data("data")

In [33]:
def compute_T(alpha_yz, alpha_zy, alpha_zx):
    """Compute inverse misalignment matrix T⁻¹"""
    return np.array([
        [1, alpha_yz, -alpha_zy],
        [0, 1, alpha_zx],
        [0, 0, 1]
    ])

def compute_R(phi, varphi):
    """Compute rotation matrix R."""
    return np.array([
        [np.cos(varphi)*np.cos(phi), np.sin(varphi)*np.cos(phi), -np.sin(phi)],
        [-np.sin(varphi), np.cos(varphi), 0],
        [np.cos(varphi)*np.sin(phi), np.sin(varphi)*np.sin(phi), np.cos(phi)]
    ])

def model_output(theta1, u):
    """Compute predicted accelerometer output: μ = K T⁻¹ u + b."""
    kx, ky, kz, alpha_yz, alpha_zy, alpha_zx, bx, by, bz = theta1
    K = np.diag([kx, ky, kz])
    T_inv = compute_T(alpha_yz, alpha_zy, alpha_zx)
    return K @ T_inv @ u + np.array([bx, by, bz])

def orientation_residuals(theta2_i, theta1, data_i):
    """Residuals for Euler angles (phi, varphi) of one orientation."""
    phi, varphi = theta2_i
    # Simplified rotation matrix (assuming heading ψ=0)
    R = compute_R(phi, varphi)
    u = R @ np.array([0, 0, g])  # True specific force
    residuals = data_i - model_output(theta1, u)
    return residuals.flatten()

def calibration_residuals(theta1, theta2, data):
    """Residuals for calibration parameters (theta1)."""
    residuals = []
    for m in range(M):
        phi, varphi = theta2[2*m], theta2[2*m+1]
        R = compute_R(phi, varphi)
        u = R @ np.array([0, 0, g])
        y_pred = model_output(theta1, u)
        residuals.extend(data[m] - y_pred)
    return np.array(residuals).flatten()

# Applying methods

In [34]:
# Initialize parameters
theta1 = np.array([1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])  # [kx, ky, kz, α_yz, α_zy, α_zx, bx, by, bz]
theta2 = np.zeros(2 * M)  # [phi_1, varphi_1, ..., phi_M, varphi_M]

# Iterative calibration
max_iterations = 20000
tolerance = 1e-6
for iteration in range(max_iterations):
    # Step 1: Estimate orientations (theta2)
    for m in range(M):
        result = least_squares(
            orientation_residuals,
            theta2[2*m:2*m+2],
            args=(theta1, data[m]),
            bounds=([-np.pi/2, -np.pi], [np.pi/2, np.pi])
        )
        theta2[2*m:2*m+2] = result.x
    
    # Step 2: Estimate calibration parameters (theta1)
    result = least_squares(
        calibration_residuals,
        theta1,
        args=(theta2, data)
    )
    theta1_new = result.x
    
    # Check convergence
    if np.linalg.norm(theta1_new - theta1) < tolerance:
        print(f"Converged after {iteration+1} iterations.")
        theta1 = theta1_new
        break
    theta1 = theta1_new
else:
    print("Max iterations reached without convergence.")

# Output results
print("\nCalibrated Parameters:")
print(f"Scale factors: kx = {theta1[0]:.6f}, ky = {theta1[1]:.6f}, kz = {theta1[2]:.6f}")
print(f"Misalignments: α_yz = {theta1[3]:.6f}, α_zy = {theta1[4]:.6f}, α_zx = {theta1[5]:.6f}")
print(f"Biases: bx = {theta1[6]:.6f}, by = {theta1[7]:.6f}, bz = {theta1[8]:.6f}")
# Calculate calibrated data using the model_output function
calibrated_data = np.zeros_like(data)
for i in range(len(data)):
    # Get orientation angles for this data point
    m = i // N
    phi, varphi = theta2[2*m], theta2[2*m+1]
    
    # Calculate true specific force
    R = compute_R(phi, varphi)
    u = R @ np.array([0, 0, g])
    
    # Apply calibration
    calibrated_data[i] = model_output(theta1, u)
    print(data[i],calibrated_data[i])
# # Create 3D scatter plot
# fig = plt.figure(figsize=(12, 8))
# ax = fig.add_subplot(111, projection='3d')

# # Plot raw data in red
# ax.scatter(data[:, 0], data[:, 1], data[:, 2], c='red', label='Raw', alpha=0.6, s=10)

# # Plot calibrated data in blue
# ax.scatter(calibrated_data[:, 0], calibrated_data[:, 1], calibrated_data[:, 2], 
#           c='blue', label='Calibrated', alpha=0.6, s=10)

# # Add labels and title
# ax.set_xlabel('X Acceleration (m/s²)')
# ax.set_ylabel('Y Acceleration (m/s²)')
# ax.set_zlabel('Z Acceleration (m/s²)')
# ax.set_title('Raw vs Calibrated Accelerometer Data')
# ax.legend()

# plt.show()

Converged after 180 iterations.

Calibrated Parameters:
Scale factors: kx = -0.054710, ky = 0.832050, kz = 0.959363
Misalignments: α_yz = 0.001567, α_zy = 31.838816, α_zx = 0.680041
Biases: bx = 0.794756, by = 0.027172, bz = -0.920688
[ 0.26  0.03 -0.92] [ 0.26035477  0.02792137 -0.91941821]
[ 0.26  0.03 -0.92] [ 0.26035477  0.02792137 -0.91941821]
[ 0.26  0.02 -0.92] [ 0.26035477  0.02792137 -0.91941821]
[ 0.27  0.03 -0.91] [ 0.26035477  0.02792137 -0.91941821]
[ 0.26  0.03 -0.91] [ 0.26035477  0.02792137 -0.91941821]
[ 0.26  0.03 -0.92] [ 0.26035477  0.02792137 -0.91941821]
[ 0.27  0.03 -0.92] [ 0.26035477  0.02792137 -0.91941821]
[ 0.26  0.03 -0.92] [ 0.26035477  0.02792137 -0.91941821]
[ 0.27  0.03 -0.92] [ 0.26035477  0.02792137 -0.91941821]
[ 0.26  0.03 -0.92] [ 0.26035477  0.02792137 -0.91941821]
[ 0.26  0.03 -0.93] [ 0.26035477  0.02792137 -0.91941821]
[ 0.26  0.03 -0.91] [ 0.26035477  0.02792137 -0.91941821]
[ 0.26  0.03 -0.93] [ 0.26035477  0.02792137 -0.91941821]
[ 0.26  0.0

# Filtering data

In [39]:
def filter_data(raw_data):
    """Apply a simple moving average filter."""
    window_size = 8
    filtered_data_x = np.convolve(raw_data[:, 0], np.ones(window_size)/window_size, mode='valid')
    filtered_data_y = np.convolve(raw_data[:, 1], np.ones(window_size)/window_size, mode='valid')
    filtered_data_z = np.convolve(raw_data[:, 1], np.ones(window_size)/window_size, mode='valid')
    return np.column_stack((filtered_data_x, filtered_data_y, filtered_data_z))

# Double Integration for Positioning

In [ ]:
def integrate_data(filtered_data):
    """Integrate accelerometer data to get velocity and position."""
    dt = 1/200  # Sampling interval (200 Hz)
    velocity = np.cumsum(filtered_data, axis=0) * dt
    position = np.cumsum(velocity, axis=0) * dt
    return position , velocity

# Movement End Detection

In [ ]:
def movment_end_detection(accelartion,count):
    """Detect the end of movement based on acceleration."""
    threshold = 0.1  # Threshold for movement end detection
    end_indices = np.where(np.linalg.norm(accelartion, axis=1) < threshold)[0]
    if len(end_indices) == 0:
        return len(accelartion)
    return end_indices[0]